In [1]:
import tensorflow as tf
import numpy as np
from numpy import genfromtxt
import os
import matplotlib.pyplot as plt
import matplotlib

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

def predict(file):
    def normalization_2d(data):
        for i in range(len(data)):
            for j in range(6):
                data[i][j] = (data[i][j]-raw_mean)/raw_std
        return data

    def normalization(data):
        for i in range(len(data)):
            data[i] = (data[i]-raw_mean)/raw_std
        return data

    def denormalization(data):
        for i in range(len(data)):
             data[i] = (data[i] * raw_std)+raw_mean
        return data

    def denormalization_2d(data):
        for i in range(len(data)):
            for j in range(6):
                data[i][j] = (data[i][j] * raw_std)+raw_mean
        return data

    train = genfromtxt('train.csv', delimiter=',', skip_header=1, usecols=(1, 2, 3, 4, 5, 6))
    label = genfromtxt('train.csv', delimiter=',', skip_header=1, usecols=(7))
    raw = genfromtxt('train.csv', delimiter=',', skip_header=1, usecols=(1))

    # 예측해야할 test
    test = genfromtxt('train.csv', delimiter=',', skip_header=1, usecols=(1,2,3,4,5,6))
    # 예측한 결과를 담아야 할 result (test에서 구한거 denormalization 해야 함.)

    raw_mean = np.mean(raw)
    raw_std = np.std(raw)
    print(np.mean(raw), np.std(raw))

    # normalization
    normalization_2d(train)
    normalization(label)
    normalization(test)

    # Data Split (train : 70%, test : 30 %) /
    # IDEA : Hyper-parameter 정할 땐 valid set 활용 하고 최종 학습 할 땐 train 100 다 사용 하기.
    train_size = int(len(train) * 0.9)
    print("Train_Size :", train_size)
    test_size = len(train) - train_size
    trainX, validX = np.array(train[0:train_size]), np.array(train[train_size:len(train)])
    trainY, validY = np.array(label[0:train_size]), np.array(label[train_size:len(train)])

    # train Parameters
    seq_length = 6  # 6일이라서 seq_length는 6이다.
    data_dim = 1  # 6일의 통으로 되어있는 data가 온도라는 피쳐 1개로 이루어져있음을 뜻함.
    # 이전 hidden_dim : 10 40 160 640 1000
    hidden_dim = 512
    output_dim = 1
    learning_rate = 0.001
    # 이전 iterations = 200 400 800 1600
    iterations = 60

    trainX_Li = []
    trainY_Li = []
    for i in range(0, len(trainY) - seq_length):
        _x = trainX[i]
        _y = trainY[i + seq_length]
        #     print(_x, "->", _y)
        trainX_Li.append(_x)
        trainY_Li.append(_y)

    trainX_Li = np.array(trainX_Li)
    trainX_Li = trainX_Li.reshape(-1, 6, 1)

    trainY_Li = np.array(trainY_Li)
    trainY_Li = trainY_Li.reshape(-1, 1)

    # 2) Valid Dataset
    # build a Valid Dataset for correct shpae
    validX_Li = []
    validY_Li = []
    for i in range(0, len(validY) - seq_length):
        _x = validX[i]
        _y = validY[i + seq_length]  # Next close price
        #     print(_x, "->", _y)
        validX_Li.append(_x)
        validY_Li.append(_y)

    validX_Li = np.array(validX_Li)
    validX_Li = validX_Li.reshape(-1, 6, 1)

    validY_Li = np.array(validY_Li)
    validY_Li = validY_Li.reshape(-1, 1)

    # 3) Valid Dataset
    # build a Valid Dataset for correct shpae
    test_Li = []
    for i in range(0, len(test) - seq_length):
        _x = test[i]
        #     print(_x, "->", _y)
        test_Li.append(_x)

    test_Li = np.array(test_Li)
    test_Li = test.reshape(-1, 6, 1)

    print(trainX_Li.shape)
    print(trainY_Li.shape)

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, 1])

    cells = []
    dropout = tf.placeholder(tf.float32)
    #이전 range(5)
    # for _ in range(3):
    rnn_cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim)
    #     rnn_cell = tf.contrib.rnn.DropoutWrapper(rnn_cell, output_keep_prob=1.0 - 0.3)
    # cells.append(rnn_cell)
    # rnn_cell = tf.contrib.rnn.MultiRNNCell(cells)

    outputs, _states = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(
        outputs[:, -1], output_dim, activation_fn=None, normalizer_fn = tf.contrib.slim.batch_norm)  # We use the last cell's output

    # cost/loss
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # mean of the squares
    # optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    # RMSE
    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])
    rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

    # print(validX_Li)
    losses = []
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        # Training step
        for i in range(iterations):
            _, step_loss = sess.run([train, loss], feed_dict={
                                    X: trainX_Li, Y: trainY_Li})
            print("[step: {}] loss: {}".format(i, step_loss))
            losses.append(step_loss)
        # Test step
        train_predict = sess.run(Y_pred, feed_dict={X: trainX_Li})
        test_predict = sess.run(Y_pred, feed_dict={X: validX_Li})
        # inverse normalize
        train_predict = denormalization(train_predict)
        test_predict = denormalization(test_predict)
        trainY_Li = denormalization(trainY_Li)
        print('valid',validY_Li)
        #RMSE
        rmse_val_tr = sess.run(rmse, feed_dict={
                        targets: trainY_Li, predictions: train_predict})
        print("TRAIN_RMSE: {}".format(rmse_val_tr))
        rmse_val = sess.run(rmse, feed_dict={
                        targets: validY_Li, predictions: test_predict})
        print("TEST_RMSE: {}".format(rmse_val))
    #     과제 제출용 step
        test_predict = sess.run(Y_pred, feed_dict={X:test_Li})

        print('complete')

        finalResult = []
        for i in test_predict:
            finalResult.append(i[0])
        denormalization(finalResult)
        return finalResult

def write_result(predictions):
    # You don't need to modify this function.
    with open('result.csv', 'w') as f:
        f.write('Value\n')
        for l in predictions:
            f.write('{}\n'.format(l))


def main():
    # You don't need to modify this function.
    predictions = predict('test.csv')
    write_result(predictions)
if __name__ == '__main__':
    # You don't need to modify this part.
    main()

C:\Users\Genius Scene\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\Genius Scene\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Genius Scene\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Genius Scene\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Genius Scene\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Genius Scene\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    io

OSError: train.csv not found.